**<center><h1>Introduction</h1></center>**

In Azure Databricks, data scientists use DataFrames to structure their data. A DataFrame is equivalent to a relational table in Spark SQL. In this module, you will learn what [DataFrames](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html#pyspark.sql.DataFrame?azure-portal=true) are and how to use them.


**<h2>Learning Objectives</h2>**

After completing this module, you’ll be able to:

- Describe DataFrames.
- Query DataFrames.
- Visualize data with Spark.

<hr>

**<center><h1>Understand dataframes</h1></center>**

Spark uses 3 different APIs: RDDs, DataFrames, and DataSets. The architectural foundation is the resilient distributed dataset (RDD). The DataFrame API was released as an abstraction on top of the RDD, followed later by the Dataset API. We'll only use DataFrames in our notebook examples.

DataFrames are the distributed collections of data, organized into rows and columns. Each column in a DataFrame has a name and an associated type.

Spark DataFrames can be created from various sources, such as CSV files, JSON, Parquet files, Hive tables, log tables, and external databases.


**<h2>Using Spark to load table data</h2>**

Assuming we have this data available in a table:

<img src="images/01-02-01-select-sql.png" />

We can use Spark to load the table data by using the sql method:
```
df = spark.sql("SELECT * FROM nyc_taxi_csv")
```
Spark supports many different data formats, such as CSV, JSON, XML, Parquet, Avro, ORC and more.


**<h2>DataFrame size</h2>**

To get the number of rows available in a DataFrame, we can use the ```count()``` method.
```
df.count
```

**<h2>DataFrame structure</h2>**

To get the schema metadata for a given DataFrame, we can use the ```printSchema()``` method.

Each column in a given DataFrame has a name, a type, and a nullable flag.
```
df.printSchema
```

<img src="images/01-02-04-spark-dataframe-printschema.png" />

**<h2>DataFrame contents</h2>**

Spark has a built-in function that allows to print the rows inside a DataFrame: ```show()```
```
df.show
df.show(100, truncate=False) #show more lines, do not truncate
```
By default it will only show the first 20 lines in your DataFrame and it will truncate long columns. Additional parameters are available to override these settings.





<hr>

**<center><h1>Query dataframes</h1></center>**

DataFrames allow the processing of huge amounts of data. Spark uses an optimization engine to generate logical queries. Data is distributed over your cluster and you get huge performance for massive amounts of data.

Spark SQL is a component that introduced the DataFrames, which provides support for structured and semi-structured data.

Spark has multiple interfaces (APIs) for dealing with DataFrames:

- We have seen the .sql() method, which allows to run arbitrary SQL queries on table data.
- Another option is to use the Spark domain-specific language for structured data manipulation, available in Scala, Java, Python, and R.


**<h2>DataFrame API</h2>**

The Apache Spark DataFrame API provides a rich set of functions (select columns, filter, join, aggregate, and so on) that allow you to solve common data analysis problems efficiently.

A complex operation where tables are joined, filtered, and restructured is easy to write, easy to understand, type safe, feels natural for people with prior sql experience, and comes with the added speed of parallel processing given by the Spark engine.

To load or save data use read and ```write```:
```
df = spark.read.format('json').load('sample/trips.json')
df.write.format('parquet').bucketBy(100, 'year', 'month').mode("overwrite").saveAsTable('table1'))
```
To get the available data in a DataFrame use ```select```:
```
df.select('*')
df.select('tripDistance', 'totalAmount')
```
To extract the first rows, use ```take```:
```
df.take(15)
```
To order the data, use the sort ```method```:
```
df.sort(df.tripDistance.desc())
```
To combine the rows in multiple DataFrames use ```union```:
```
df1.union(df2)
```
This operation is equivalent to ```UNION ALL``` in SQL. To do a SQL-style set union (that does deduplication of elements), use this function followed by ```distinct()```.

The dataframes must have the same structure/schema.

To add or update columns use ```withColumn``` or ```withColumnRenamed```:
```
df.withColumn('isHoliday', False)
df.withColumnRenamed('isDayOff', 'isHoliday')
```
To use aliases for the whole DataFrame or specific columns:
```
df.alias("myTrips")
df.select(df.passengerCount.alias("numberOfPassengers"))
```

To create a temporary view:
```
df.createOrReplaceTempView("tripsView")
```
To aggregate on the entire DataFrame without groups use ```agg```:
```
df.agg({"age": "max"})
```

To do more complex queries, use ```filter```, ```groupBy``` and ```join```:

```
people \
  .filter(people.age > 30) \
  .join(department, people.deptId == department.id) \
  .groupBy(department.name, "gender")
  .agg({"salary": "avg", "age": "max"})

```
These join types are supported: inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, and left_anti.

Note that ```filter``` is an alias for ```where```.

To use columns aggregations using windows:

```
w = Window.partitionBy("name").orderBy("age").rowsBetween(-1, 1)
df.select(rank().over(w), min('age').over(window))
```
To use a list of conditions for a column and return an expression use ```when```:

```
df.select(df.name, F.when(df.age > 4, 1).when(df.age < 3, -1).otherwise(0)).show()
```

To check the presence of data use ```isNull``` or ```isNotNull```:

```
df.filter(df.passengerCount.isNotNull())
df.filter(df.totalAmount.isNull())
```
To clean the data use ```dropna```, ```fillna``` or ```dropDuplicates```:

```
df1.fillna(1) #replace nulls with specified value
df2.dropna #drop rows containing null values
df3.dropDuplicates #drop duplicate rows
```
To get statistics about the DataFrame use ```summary``` or ```describe```:

```
df.summary().show()
df.summary("passengerCount", "min", "25%", "75%", "max").show()
df.describe(['age']).show()
```

Available statistics are:

- Count
- Mean
- Stddev
- Min
- Max
- Arbitrary approximate percentiles specified as a percentage (for example, 75%).

To find correlations between specific columns use corr. This operation currently only supports the Pearson Correlation Coefficient:

```
df.corr('tripDistance', 'totalAmount')
```
<mark>**Note: More information: for more information about the Spark API, see the [DataFrame API](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame?azure-portal=true) and the [Column API](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.Column?azure-portal=true) in the Spark documentation.</mark>






<hr>

**<center><h1>Visualize data</h1></center>**

Spark has a built-in ```show``` function, which allows to print the rows in a DataFrame.

Azure Databricks adds its own display capabilities and adds various other types of visualizations out-of-the-box using the ```display``` and ```displayHTML``` functions.

The same data we've seen previously as a table can be displayed as a bar chart, pie, histogram, or other graphs. Even maps or images can be displayed:

<img src="images/01-02-07-spark-dataframe-view.png" />


**<h2>Plot options</h2>**

The following display options are available:

- We can choose the DataFrame columns to be used as axes (keys, values).
- We can choose to group our series of data.
- We can choose the aggregations to be used with our grouped data (avg, sum, count, min, max).

<img src="images/01-02-08-spark-dataframe-view-plot-options.png" />

<mark>**Note:** More information: for more information about the available visualizations, see Visualizations in the Azure Databricks documentation.</mark>


<hr>

**<center><h1>Exercise - Work with data in Azure Databricks</h1></center>**

Now it's your chance to experience Azure Databricks for yourself by loading data, manipulating it and visualizing the results.

In this exercise, you will:

- Load data into a DataFrame.
- Query a dataframe.
- Transform data using DataFrames.
- Visualize data.

**<h2>Instructions</h2>**
Follow these instructions to complete the exercise:

1. Open the exercise instructions at https://aka.ms/mslearn-dp090.
2. Complete the Working with **Data in Azure Databricks** exercise.



<hr>

**<center><h1>Summary</h1></center>**

In this module, you have learned how to work with data on Azure Databricks.

Now that you've completed this module, you can:

- Describe DataFrames.
- Query DataFrames.
- Visualize data with Spark.



<hr>